# Importing the necessary libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from zipfile import ZipFile
# !pip install plotly
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.cluster import KMeans
import plotnine as pn
import plydata.cat_tools as cat
# machine learning
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# Importing the lifetimes package
import lifetimes
from lifetimes.plotting import plot_probability_alive_matrix
from sklearn.metrics import mean_absolute_error

# Loading the Data

In [ ]:
# Laoding the data into a dataframe.
CDNOW_DATA = pd.read_csv("Data/CDNOW_master.txt", sep='\s+')
CDNOW_DATA.shape

In [ ]:
CDNOW_DATA.head() # Looking at the top 5 rows of the dataframe.

# Data Cleaning

In [ ]:
CDNOW_DATA.info() # info() gives us the number of non null values and the data type of each column.

In [ ]:
CDNOW_DATA.columns = ["CustomerID", "TransactionDate", "UnitsSold", "Price"] # Assigning the column names to the data frame.

CDNOW_DATA["CustomerID"] = CDNOW_DATA["CustomerID"].astype(str) # changing the datatype of the CustomerID to string.

# changing the datatye of the TransactionDate column to datetime.
CDNOW_DATA['TransactionDate'] = CDNOW_DATA['TransactionDate'].apply(lambda x: pd.to_datetime(str(x), format='%Y-%m-%d'))

In [ ]:
CDNOW_DATA.head()  # Looking at the top 5 rows of the dataframe.

# Descriptive Analytics

In [ ]:
CDNOW_DATA.describe() # looking at the min, max, mean, standard deviation and quartile values of the numerical values.

In [ ]:
# First Transaction Date
print("First Transaction Date", CDNOW_DATA["TransactionDate"].min())

# Last Transaction Date
print("Last Transaction Date", CDNOW_DATA["TransactionDate"].max())

# Number of Unique Customers
print("Number of Unique Customers", CDNOW_DATA["CustomerID"].nunique())

# Total Revenue.
print("Total Revenue", CDNOW_DATA["Price"].sum())

### Customers with highest number of Transactions

In [ ]:
# Grouping the data to create a dataframe with CustomerID and number of transactions that customer made.
NO_OF_TRANSACTIONS = CDNOW_DATA.groupby("CustomerID")["UnitsSold"].count() \
                                        .reset_index().sort_values("UnitsSold", ascending = False) \
                                        .rename(columns = {"UnitsSold" : "NO_OF_TRANSACTIONS"}) \
                                        .reset_index(drop = True) 


TOP_10_HIGHEST_TRANSACTIONS = NO_OF_TRANSACTIONS.iloc[:10, :2] # slicing to get just the 10 rows worth of data.
TOP_10_HIGHEST_TRANSACTIONS["CustomerID"] = TOP_10_HIGHEST_TRANSACTIONS["CustomerID"].astype("str")
TOP_10_HIGHEST_TRANSACTIONS.shape

In [ ]:
# plt.bar() function is used for plotting the bar plot.
# plt.ylabel() function is used for labelling the Y-axis.
# plt.xlabel() function is used for labelling the X-axis.
# plt.title() function is used for setting the title for the plot.
# plt.legend() function is used for setting the legend for the plot
# plt.show() function is used for displaying the plot.

        
#declaring the legend for the plot.
colors = {'CUSTOMER WITH HIGHEST NUMBER OF TRANSACTIONS':'green'}         
labels = list(colors.keys())
handles = [plt.Rectangle((0,0),1,1, color=colors[label]) for label in labels]
plt.legend(handles, labels, bbox_to_anchor=(1.05, 1))

clrs1=[]
# declaring the colors of the bar plot's bars.
for i in TOP_10_HIGHEST_TRANSACTIONS['NO_OF_TRANSACTIONS']:
    if(i==max(TOP_10_HIGHEST_TRANSACTIONS['NO_OF_TRANSACTIONS'])):
        clrs1.append('green')
    else:
        clrs1.append('blue')
        

#Plot
bar_plot=plt.bar(TOP_10_HIGHEST_TRANSACTIONS['CustomerID'], TOP_10_HIGHEST_TRANSACTIONS['NO_OF_TRANSACTIONS'], color=clrs1)
plt.title('10 CUSTOMERS WITH HIGHEST TRANSACTIONS', pad =25,fontweight='bold', fontname="Times New Roman", style='italic')
plt.ylabel('NUMBER OF TRANSACTIONS', fontweight='bold', color = 'Black', verticalalignment='center', labelpad=30, fontname="Times New Roman", style='italic') # ylabel function is used for labelling the Y-axis.
plt.xlabel('CUSTOMER ID',fontweight='bold', color = 'Black', horizontalalignment='center', labelpad=30, fontname="Times New Roman", style='italic') # xlabel function is used for labelling the X-axis.
plt.xticks(rotation='vertical')


plt.show()

In [ ]:
MONTHLY_INCOME = pd.DataFrame(CDNOW_DATA.set_index('TransactionDate')["Price"].resample("M").sum())
MONTHLY_INCOME.head()

# Plotting the bar plot of Transaction_data and Price.

fig = go.Figure([go.Scatter(x=MONTHLY_INCOME.index, y=MONTHLY_INCOME['Price'])])
fig.update_layout(
    title="MONTHLY INCOME",
    xaxis_title="DATE",
    yaxis_title="INCOME",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#7f7f7f"
    ),
)
fig.show()
# Reference:
# https://plotly.com/python/time-series/
# https://plotly.com/python/reference/layout/

### Cohort Analysis

In [ ]:
# Customers first transaction. Considering the first transaction of the customers as a subset to perfrom our analysis.
CDNOW_FIRST_TRANSACTION = CDNOW_DATA.sort_values(["CustomerID", "TransactionDate"]) \
                               .groupby("CustomerID").first()
CDNOW_FIRST_TRANSACTION.shape

In [ ]:
# first new customer.
CDNOW_FIRST_TRANSACTION["TransactionDate"].min()

In [ ]:
# last new customer
CDNOW_FIRST_TRANSACTION["TransactionDate"].max()

In [ ]:
# GETTING THE UNIQUE CUSTOMERS ID's
IDs = CDNOW_DATA['CustomerID'].unique()
IDs_SAMPLE = IDs[0:10]

In [ ]:
# Creating a subset of the dataset with ten unique customer IDs
CDNOW_SUBSET = CDNOW_DATA[CDNOW_DATA['CustomerID'].isin(IDs_SAMPLE)].groupby(['CustomerID', 'TransactionDate']).sum().reset_index()
CDNOW_SUBSET.head()

In [ ]:
# Frequency plot of the ten unique customers, with the price and date of each transaction.
pn.ggplot(pn.aes('TransactionDate', 'Price', group = 'CustomerID'), data = CDNOW_SUBSET) + pn.geom_line() +pn.geom_point() +pn.facet_wrap('CustomerID') +pn.scale_x_date(date_breaks = '1 year', date_lables = '%Y')

# Customer Segmentation <br> <i> RFM analysis</i>

### Recency

In [ ]:
# MOST RECENT TRANSACTIONS OF THE CUSTOMERS 
# RFM ("R": Recency)

CDNOW_Recency = CDNOW_DATA.groupby('CustomerID', as_index=False)['TransactionDate'].max()
CDNOW_Recency.columns = ['CustomerID', 'LAST_TransactionDate']
RECENT_TransactionDate = CDNOW_Recency['LAST_TransactionDate'].max()
CDNOW_Recency['Recency'] = CDNOW_Recency['LAST_TransactionDate'].apply(lambda x: (RECENT_TransactionDate - x).days)
CDNOW_Recency.head()

In [ ]:
CDNOW_Recency["Recency"].describe() # Gives the min, max and mean values of the Recency.

### Frequency

In [ ]:
# CALCULACTING THE Frequency OF THE CUSTOMERS
# RFM ("F": Frequency)

CDNOW_Frequency = CDNOW_DATA.drop_duplicates().groupby('CustomerID', as_index=False)['TransactionDate'].count()
CDNOW_Frequency.columns = ['CustomerID', 'Frequency']
CDNOW_Frequency.head()

In [ ]:
CDNOW_Frequency["Frequency"].describe() # Gives the min, max and mean values of the Frequency.

### Monetary

In [ ]:
# CALCULATING THE Monetary VALUE OF CUSTOMER SPEND ON PURCHASING CD's FROM THE CDNOW STORE.
# RFM ("M": Monetary)

CDNOW_Monetary = CDNOW_DATA.groupby('CustomerID', as_index=False)['Price'].sum()
CDNOW_Monetary.columns = ['CustomerID', 'Monetary']
CDNOW_Monetary.head()

CDNOW_Monetary["Monetary"].describe() # Gives the min, max and mean values of the Monetary.

### RFM

In [ ]:
# MERGING Recency Frequency AND Monetary
CDNOW_RF = CDNOW_Recency.merge(CDNOW_Frequency, on='CustomerID')
CDNOW_RFM = CDNOW_RF.merge(CDNOW_Monetary, on='CustomerID').drop(columns='LAST_TransactionDate')
CDNOW_RFM.head()

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
fig, axes = plt.subplots(1,3, figsize=(18, 5))
sns.distplot(CDNOW_RFM["Recency"], ax = axes[0])
sns.distplot(CDNOW_RFM["Frequency"], ax = axes[1])
sns.distplot(CDNOW_RFM["Monetary"], ax = axes[2])
plt.show()

# From the plots below we can see that the data has skewness and we need to normalize the data.

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(18, 5))
sns.boxplot(x = CDNOW_RFM["Recency"], ax = axes[0])
sns.boxplot(x = CDNOW_RFM["Frequency"], ax = axes[1])
sns.boxplot(x = CDNOW_RFM["Monetary"], ax = axes[2])
plt.show()
# From looking at the plot below we say that the data has some outliers.

### Robust Scaler

In [ ]:
# Normalizing the data using Robust Scalar to deal with the coutliers.
scaler = RobustScaler()
scaler.fit(CDNOW_RFM[['Recency', 'Frequency', 'Monetary']])
CDNOW_RFM[['Recency_NORM', 'Frequency_NORM', 'Monetary_NORM']] = scaler.transform(CDNOW_RFM[['Recency', 'Frequency', 'Monetary']])

fig, axes = plt.subplots(1,3, figsize=(18, 5))
sns.distplot(CDNOW_RFM["Recency_NORM"], ax = axes[0])
sns.distplot(CDNOW_RFM["Frequency_NORM"], ax = axes[1])
sns.distplot(CDNOW_RFM["Monetary_NORM"], ax = axes[2])
plt.show()

### K-means for customer segmentation.

In [ ]:
# Using the Unsupervised Learning to cluster the customers into segemets

cluster_range = range( 1, 10 )
cluster_errors = {"R_cluster_errors":[], "F_cluster_errors":[], "M_cluster_errors":[]}

In [ ]:
# Identifing the optimal number of clusters using telbow curve.
for num_clusters in cluster_range:
    R_clusters = KMeans(num_clusters)
    F_clusters = KMeans(num_clusters)
    M_clusters = KMeans(num_clusters)
    R_clusters.fit(CDNOW_RFM[['Recency_NORM']])
    F_clusters.fit(CDNOW_RFM[['Frequency_NORM']])
    M_clusters.fit(CDNOW_RFM[['Monetary_NORM']])
    cluster_errors["R_cluster_errors"].append(R_clusters.inertia_)
    cluster_errors["F_cluster_errors"].append(F_clusters.inertia_)
    cluster_errors["M_cluster_errors"].append(M_clusters.inertia_)
    

In [ ]:
# Creating a dataframe of the Cluster Errors to find the optimal Cluster threshold.
CLUSTER_ERRORS_DF = pd.DataFrame({"NO_OF_CLUSTERS":cluster_range, "R_CLUSTER_ERRORS": cluster_errors["R_cluster_errors"], 
                                  "F_CLUSTER_ERRORS": cluster_errors["F_cluster_errors"], 
                                 "M_CLUSTER_ERRORS": cluster_errors["M_cluster_errors"]})

CLUSTER_ERRORS_DF

In [ ]:
# plotting the line graph to picture the elbow curve and determine the optimal number of clusters.
plt.rcParams.update(plt.rcParamsDefault)
fig, axes = plt.subplots(1,3, figsize=(18, 5))
axes[0].plot( CLUSTER_ERRORS_DF.NO_OF_CLUSTERS, CLUSTER_ERRORS_DF.R_CLUSTER_ERRORS, marker = "o", axes = axes[0])
axes[0].set_title("Recency")
axes[0].set_xlabel("Number of clusters")
axes[0].set_ylabel("Cluster Error")
axes[1].plot( CLUSTER_ERRORS_DF.NO_OF_CLUSTERS, CLUSTER_ERRORS_DF.F_CLUSTER_ERRORS, marker = "o", axes = axes[1])
axes[1].set_title("Frequency")
axes[1].set_xlabel("Number of clusters")
axes[1].set_ylabel("Cluster Error")
axes[2].plot( CLUSTER_ERRORS_DF.NO_OF_CLUSTERS, CLUSTER_ERRORS_DF.M_CLUSTER_ERRORS, marker = "o", axes = axes[2])
axes[2].set_title("Monetary")
axes[2].set_xlabel("Number of clusters")
axes[2].set_ylabel("Cluster Error")
plt.subplots_adjust(left  = 0.05,
right = 0.9,
bottom = 0.1,
top = 0.9,
wspace = 0.2,
hspace = 1)
plt.show()

# From the Elbow Curves below:
# FOR Recency_NORM optimal clusters = 3
# FOR Frequency_NORM optimal clusters = 4
# FOR Monetary_NORM optimal clusters = 4

### K-means

In [ ]:
# Creating the clusters.

R_clusters_Optimized = KMeans(3, random_state=42)
F_clusters_Optimized = KMeans(4, random_state=42)
M_clusters_Optimized = KMeans(4, random_state=42)

In [ ]:
# fitting the data
R_clusters_Optimized.fit(CDNOW_RFM[['Recency_NORM']])
F_clusters_Optimized.fit(CDNOW_RFM[['Frequency_NORM']])
M_clusters_Optimized.fit(CDNOW_RFM[['Monetary_NORM']])

In [ ]:
# Storing the output/labels of the datapoints.
CDNOW_RFM["R_CLUSTER"] = R_clusters_Optimized.labels_
CDNOW_RFM["F_CLUSTER"] = F_clusters_Optimized.labels_
CDNOW_RFM["M_CLUSTER"] = M_clusters_Optimized.labels_

In [ ]:
# Count plots to see the count of datapoints in each cluster.
sns.countplot(CDNOW_RFM["R_CLUSTER"])
plt.show()

In [ ]:
# Count plots to see the count of datapoints in each cluster.
sns.countplot(CDNOW_RFM["F_CLUSTER"])
plt.show()

In [ ]:
# Count plots to see the count of datapoints in each cluster.
sns.countplot(CDNOW_RFM["M_CLUSTER"])
plt.show()

In [ ]:
CDNOW_RFM.head() # looking at the cluster values.

In [ ]:
# For a Customer to be a High-Value Customer, the customers:
# Recency should be low
# Frequency should be high
# Monetary Value should be high
CDNOW_RFM.sort_values('Recency_NORM', ascending = True, inplace = True)
CDNOW_RFM.sort_values('Frequency_NORM', ascending = False, inplace = True)
CDNOW_RFM.sort_values('Monetary_NORM', ascending = False, inplace = True)
CDNOW_RFM.head()

In [ ]:
# Creating the final cluster using the independent clusters.
CDNOW_RFM["FinalCluster"] = CDNOW_RFM["R_CLUSTER"] + CDNOW_RFM["F_CLUSTER"] + CDNOW_RFM["M_CLUSTER"]
CDNOW_RFM.reset_index(inplace = True, drop = True)
CDNOW_RFM.head()

In [ ]:
CDNOW_RFM.tail()

In [ ]:
plt.plot(CDNOW_RFM.index, CDNOW_RFM["FinalCluster"])
plt.show()

In [ ]:
# Naming the clusters.

CDNOW_RFM['Segment'] = 'Lost Customer'
CDNOW_RFM.loc[CDNOW_RFM['FinalCluster']==1,'Segment'] = 'Low-Value' 
CDNOW_RFM.loc[CDNOW_RFM['FinalCluster'] == 2,'Segment'] = 'Mid-Value'
CDNOW_RFM.loc[CDNOW_RFM['FinalCluster']>2,'Segment'] = 'High-Value'
CDNOW_RFM.loc[CDNOW_RFM['FinalCluster']>5,'Segment'] = 'Top Customer'

In [ ]:
sns.countplot(CDNOW_RFM['Segment'], order= ['Lost Customer','Low-Value' ,'Mid-Value','High-Value','Top Customer'])
plt.title("RFM Segmentation of CDNOW dataset")
plt.show()

In [ ]:
CDNOW_RFM.to_csv("CDNOW_CustomerSegmentation.csv", index = False)

# Machine Learning Modelling

### Feature Engineering

In [ ]:
n_days = 90 # The no of days for which we want to predict CLV.
max_date = CDNOW_DATA['TransactionDate'].max() # Getting the max transaction date.
cutoff = max_date - pd.to_timedelta(n_days, unit = 'd') # Creating a cut off to exclude the last 90 days.

IN_DATA = CDNOW_DATA[CDNOW_DATA['TransactionDate'] <= cutoff] # Excluding the last 90 days.
OUT_DATA = CDNOW_DATA[CDNOW_DATA['TransactionDate'] > cutoff] # Just the last 90 days data.

# making targets from out data
TARGET_DATA = OUT_DATA.drop('UnitsSold', axis = 1).groupby('CustomerID').sum().rename({'Price':'spend_90_total'}, axis = 1).assign(spend_90_flag = 1)
TARGET_DATA.head()

In [ ]:
# make recency (date) features from in data
max_date = IN_DATA['TransactionDate'].max()

In [ ]:
# Getting the recency data to include that as feature for ML modelling
recency_features_df = IN_DATA[['CustomerID', 'TransactionDate']].groupby('CustomerID').apply(lambda x: (x['TransactionDate'].max() - max_date)/ pd.to_timedelta(1, 'day')).to_frame().set_axis(['recency'], axis = 1)
recency_features_df.head()

In [ ]:
# make frequency (count features from in data)
frequency_features_df = IN_DATA[['CustomerID', 'TransactionDate']].groupby('CustomerID').count().set_axis(['frequency'], axis = 1)
frequency_features_df.head()

In [ ]:
# make price (monitery features from in data)
price_features_df = IN_DATA.groupby('CustomerID').aggregate({'Price':['sum', 'mean']}).set_axis(['price_sum', 'price_mean'], axis = 1)
price_features_df.head()

In [ ]:
# combine features
features_df = pd.concat([recency_features_df, frequency_features_df, price_features_df], axis = 1).merge(TARGET_DATA, left_index = True, right_index = True, how = 'left').fillna(0)
features_df.head()

### Regression
> How much will the customers spend in the next 90 days?

### XGBOOST

In [ ]:
X = features_df[['recency', 'frequency', 'price_sum', 'price_mean']]

# next 90 days spend prediction
y_spend = features_df['spend_90_total']

# Model
xgb_reg_spec = XGBRegressor(objective='reg:squarederror', random_state=123)

# Creating a GridSearchCV to tune the hyperparameters
xgb_reg_model =GridSearchCV(estimator=xgb_reg_spec, param_grid = dict(learning_rate = [0.01, 0.1, 0.3, 0.5]), scoring='neg_mean_absolute_error', cv = 5) 

In [ ]:
# Fitting the Model
xgb_reg_model.fit(X, y_spend)

In [ ]:
xgb_reg_model.best_score_ # NAE of the best parameters model.

In [ ]:
xgb_reg_model.best_params_ # best Hyperparameter value

In [ ]:
predictions_xgb_reg_model = xgb_reg_model.predict(X) # Prediction

In [ ]:
# Merging the predictions to the data.
XGB_PRED_DATA = pd.DataFrame({"prediction_xgb" : predictions_xgb_reg_model.tolist()}).merge(X.reset_index(),left_index=True, right_index=True)
XGB_PRED_DATA.head()

In [ ]:
XGB_PRED_DATA.to_csv("XGB_reg_prediction.csv") # Writing the predictions to a csv

### Linear Regression

In [ ]:
lr = LinearRegression() # Model

parameters = {'fit_intercept':[True,False],  'copy_X':[True, False]} # Hyper parameters.
lr_reg_model = GridSearchCV(lr,parameters, cv=5, scoring='neg_mean_absolute_error')
lr_reg_model.fit(X, y_spend)

In [ ]:
lr_reg_model.best_score_ # NAE of the best parameters model.

In [ ]:
lr_reg_model.best_params_ # best Hyperparameter value

In [ ]:
predictions_lr_reg_model = lr_reg_model.predict(X) # Prediction

In [ ]:
# Merging the prediction to the X data.
LINEAR_REGRESSION_PRED_DATA = pd.DataFrame({"prediction_lr" : predictions_lr_reg_model.tolist()}).merge(X.reset_index(),left_index=True, right_index=True)

In [ ]:
LINEAR_REGRESSION_PRED_DATA.to_csv("lr_reg_prediction.csv") # Predictions to CSV

### Random Forest Regression

In [ ]:
# Hyper Parameters
param_grid = dict(n_estimators = [10, 20, 50, 100, 500, 1000], max_depth = [2, 5], max_features = ['auto', 'sqrt', 'log2'])

rf = RandomForestRegressor() # model
rf_regressor =GridSearchCV(estimator=rf, param_grid = param_grid, scoring='neg_mean_absolute_error', refit= True, cv = 5) 

rf_regressor.fit(X, y_spend) #fitting the data

In [ ]:
rf_regressor.best_score_ # NAE of the best model.

In [ ]:
rf_regressor.best_params_ # Best hyper parameters.

In [ ]:
rf_regressor.best_estimator_

In [ ]:
predictions_rf_reg_model = rf_regressor.predict(X) # Prediction

In [ ]:
# Mergeing the prediction with X data
RF_REGRESSION_PRED_DATA = pd.DataFrame({"prediction_rf" : predictions_rf_reg_model.tolist()}).merge(X.reset_index(),left_index=True, right_index=True)

In [ ]:
RF_REGRESSION_PRED_DATA.to_csv("rf_reg_prediction.csv") # Storing the Prediction data in a csv.

### Classification
> Will a customer make a purchase in next 90 days

In [ ]:
# Next 90 days spend probability
y_prob = features_df['spend_90_flag']

xgb_clf_spec = XGBClassifier(objective='binary:logistic', random_state= 123) # model

xgb_clf_model =GridSearchCV(estimator=xgb_clf_spec, param_grid = dict(learning_rate = [0.01, 0.1, 0.3, 0.5]), scoring='roc_auc', refit= True, cv = 5) 

xgb_clf_model.fit(X, y_prob) # Fitting the data

In [ ]:
xgb_clf_model.best_score_ # AUC score of the best model

In [ ]:
xgb_clf_model.best_params_ # Best Hyperparameter Values

In [ ]:
# Storing the prediction values in a dataframe
XGB_CLASSIFICATION_PRED_DATA = pd.DataFrame(xgb_clf_model.predict_proba(X),columns=['prediction_xgb_clf_0','prediction_xgb_clf_1']).merge(X.reset_index(),left_index=True, right_index=True)
XGB_CLASSIFICATION_PRED_DATA.head()

In [ ]:
XGB_CLASSIFICATION_PRED_DATA.to_csv("xgb_clf_prediction.csv") # Predictions to csv

### Logistic Regression

In [ ]:
parameters = {'penalty': ["l1", "l2"], 'solver': ["liblinear","saga"]} # Hyper parameters

logr = LogisticRegression() # model
logr_clf_model = GridSearchCV(logr,parameters, cv=5, scoring='accuracy')
logr_clf_model.fit(X, y_prob) #fitting

In [ ]:
logr_clf_model.best_params_ # Best Hyperparameters

In [ ]:
logr_clf_model.best_score_ # AUC score of the best model

In [ ]:
# Storing the prediction values in a dataframe with X data
LOGISTIC_CLASSIFICATION_PRED_DATA = pd.DataFrame(logr_clf_model.predict_proba(X),columns=['prediction_log_clf_0','prediction_log_clf_1']).merge(X.reset_index(),left_index=True, right_index=True)
LOGISTIC_CLASSIFICATION_PRED_DATA.head()

In [ ]:
LOGISTIC_CLASSIFICATION_PRED_DATA.to_csv("log_clf_prediction.csv") # Prediction to csv.

### RANDOM FOREST CLASSIFICATION

In [ ]:
#Hyper Parameters
parameters = { 'max_depth' : [2, 5], 'n_estimators' : [10, 20, 50, 100, 500, 1000],'max_features' : ['auto', 'sqrt', 'log2']}

In [ ]:
rfc = RandomForestClassifier() # mdoel
rfc_clf_model = GridSearchCV(rfc, parameters, cv=5, scoring='roc_auc') 
rfc_clf_model.fit(X, y_prob) # fitting the data

In [ ]:
rfc_clf_model.best_params_ # Best parameters

In [ ]:
rfc_clf_model.best_score_ # AUC score of the best model.

In [ ]:
# Storing the prediction values
RF_CLASSIFICATION_PRED_DATA = pd.DataFrame(rfc_clf_model.predict_proba(X),columns=['prediction_rf_clf_0','prediction_rf_clf_1']).merge(X.reset_index(),left_index=True, right_index=True)
RF_CLASSIFICATION_PRED_DATA.to_csv("rf_clf_prediction.csv") # Predictions to csv.

### Feature importance

In [ ]:
#importance | spend amount model

imp_spend_amount_dict = xgb_reg_model.best_estimator_.get_booster().get_score(importance_type = 'gain')

imp_spend_amount_df = pd.DataFrame(data = {'feature': list(imp_spend_amount_dict.keys()), 
                                   'value': list(imp_spend_amount_dict.values())}).assign(feature = lambda x : cat.cat_reorder(x['feature'], x['value']))

imp_spend_amount_df.head()

In [ ]:
pn.ggplot(pn.aes('feature', 'value'), data = imp_spend_amount_df) + pn.geom_col() + pn.coord_flip() #plot of feature importance.

In [ ]:
# Importance | Spend probability model
imp_spend_prob_dict = xgb_clf_model.best_estimator_.get_booster().get_score(importance_type = 'gain')

imp_spend_prob_df = pd.DataFrame(data = {'feature': list(imp_spend_prob_dict.keys()), 
                                   'value': list(imp_spend_prob_dict.values())}).assign(feature = lambda x : cat.cat_reorder(x['feature'], x['value']))

In [ ]:
pn.ggplot(pn.aes('feature', 'value'), data = imp_spend_prob_df) + pn.geom_col() + pn.coord_flip() #plot of feature importance.

# CLV Mathematically

In [ ]:
# Creating a customer level dataframe
customer_level_data = CDNOW_DATA.groupby("CustomerID").aggregate({"TransactionDate": lambda x: (x.max() - x.min()).days,
                                                              "CustomerID": lambda x: len(x),
                                                   "Price" : lambda x: x.sum()})
customer_level_data.columns = ['num_days', 'num_transactions', 'spent_money']
customer_level_data.head()

### AVG_ORDER_VALUE

In [ ]:
#Calculating the average order value for each customer
customer_level_data['avg_order_value']=customer_level_data['spent_money']/customer_level_data['num_transactions']
customer_level_data.head()

### PURCHASE FREQUNECY

In [ ]:
purchase_frequency=sum(customer_level_data['num_transactions'])/customer_level_data.shape[0]
purchase_frequency

### REPEAT RATE

In [ ]:
# Repeat Rate
repeat_rate=customer_level_data[customer_level_data.num_transactions > 1].shape[0]/customer_level_data.shape[0]
repeat_rate

### CHURN

In [ ]:
#Churn Rate
churn_rate=1-repeat_rate
churn_rate

In [ ]:
purchase_frequency,repeat_rate,churn_rate

### PROFIT

In [ ]:
# Assuming profit marging to be of 5%
customer_level_data["profit"] = customer_level_data["spent_money"]* 0.05
customer_level_data.head()

### CLV

In [ ]:
# CLV
customer_level_data['CLV']=(customer_level_data['avg_order_value']*purchase_frequency)/churn_rate

### CLTV

In [ ]:
customer_level_data['cust_lifetime_value']=customer_level_data['CLV']*customer_level_data['profit']
customer_level_data.head()

In [ ]:
cdnow = CDNOW_DATA.copy()

In [ ]:
cdnow['month_yr'] = cdnow['TransactionDate'].apply(lambda x: x.strftime('%b-%Y'))
cdnow.head()

In [ ]:
sale=cdnow.pivot_table(index=['CustomerID'],columns=['month_yr'],values='Price',aggfunc='sum',fill_value=0).reset_index()
sale.head()

In [ ]:
sale['CLV']=sale.iloc[:,2:].sum(axis=1)
sale.head()

In [ ]:
cdnow["TransactionDate"].max()

# CLV prediction based on the latest 6 month sales data:

>Predictive modeling on the customer lifetime value instead of the sales expected in next 90 days

In [ ]:
# Selecting features as the latest 6 month data

X2=sale[['Jan-1998','Feb-1998', 'Mar-1998','Apr-1998','May-1998','Jun-1998']]
y=sale[['CLV']]

### XGBOOST

In [ ]:
xgb = XGBRegressor(objective='reg:squarederror', random_state = 123) #model

# GridSearch with all the hyperparameters
xgb =GridSearchCV(estimator=xgb, param_grid = dict(learning_rate = [0.01, 0.1, 0.3, 0.5]), scoring='neg_mean_absolute_error', refit= True, cv = 5)

xgb.fit(X2, y) # Fitting the data

In [ ]:
xgb.best_score_ # NAE for the best model

In [ ]:
xgb.best_params_ # Hyper parameters for the best model

In [ ]:
predictions_xgb_reg_6model = xgb.predict(X2) # prediction

In [ ]:
XGBOOST_6M_REG = pd.DataFrame({"prediction_xgb_6m" : predictions_xgb_reg_6model.tolist()}).merge(X2,left_index=True, right_index=True).merge(sale["CustomerID"],left_index=True, right_index=True)

XGBOOST_6M_REG.to_csv("XGB_reg_6m_prediction.csv") # Prediction to csv.

### Linear Regression

In [ ]:
lr = LinearRegression()

parameters = {'fit_intercept':[True,False],  'copy_X':[True, False]} # Hyperparameters
lr_reg_model = GridSearchCV(lr,parameters, cv=5, scoring='neg_mean_absolute_error')
lr_reg_model.fit(X2, y) #fittig the data

In [ ]:
lr_reg_model.best_score_ # NAE for the best model

In [ ]:
lr_reg_model.best_params_ # Hyperparameters for the best model.

In [ ]:
predictions_lr_reg_6model = lr_reg_model.predict(X2).reshape(-1)
predictions_lr_reg_6model

In [ ]:
# Storing the prediction in a dataframe.
LinearReg_6M_REG = pd.DataFrame({"prediction_lr_6m" : predictions_lr_reg_6model.tolist()}).merge(X2,left_index=True, right_index=True).merge(sale["CustomerID"],left_index=True, right_index=True)

LinearReg_6M_REG.to_csv("lr_reg_6m_prediction.csv")# Prediction to csv.

### Random Forest Regression

In [ ]:
rf = RandomForestRegressor() # Model
rf_regressor =GridSearchCV(estimator=rf, param_grid = param_grid, scoring='neg_mean_absolute_error', refit= True, cv = 5) 

rf_regressor.fit(X2, y.values.ravel()) # fitting the data

In [ ]:
rf_regressor.best_score_ # NAE for the best model

In [ ]:
predictions_rf_reg_6model = rf_regressor.predict(X2) # Prediction

In [ ]:
RFReg_6M_REG = pd.DataFrame({"prediction_rf_6m" : predictions_rf_reg_6model.tolist()}).merge(X2,left_index=True, right_index=True).merge(sale["CustomerID"],left_index=True, right_index=True)

RFReg_6M_REG.to_csv("rf_reg_6m_prediction.csv")# Prediction to csv.

# Lifetimes Library

In [ ]:
CDNOW_DATA = CDNOW_DATA[CDNOW_DATA['Price'] > 0]
TRAIN = CDNOW_DATA[CDNOW_DATA["TransactionDate"] < CDNOW_DATA["TransactionDate"].max() - dt.timedelta(days = 30)]
TRAIN.shape

In [ ]:
TEST =  CDNOW_DATA[CDNOW_DATA["TransactionDate"] >= CDNOW_DATA["TransactionDate"].max() - dt.timedelta(days = 30)]
TEST.shape

In [ ]:
CDNOW_data_LT = lifetimes.utils.summary_data_from_transaction_data(TRAIN, 'CustomerID', 'TransactionDate', 'Price' )
CDNOW_data_LT.head()
# frequency: total number of repeat purchases
# recency: the difference between the customer's last purchase and his first purchase
# T: the age of the client in the company
# monetary_value: average earnings per purchase

In [ ]:
# Create a distribution of frequency to understand the customer frequency level
CDNOW_data_LT['frequency'].plot(kind='hist', bins=50)
plt.show()
print(CDNOW_data_LT['frequency'].describe())
print("---------------------------------------")
one_time_buyers = round(sum(CDNOW_data_LT['frequency'] == 0)/float(len(CDNOW_data_LT))*(100),2)
print("Percentage of customers that purchased the item only once:", one_time_buyers ,"%")

In [ ]:
# Fitting the BG/NBD model
bgf = lifetimes.BetaGeoFitter(penalizer_coef=0.0)
bgf.fit(CDNOW_data_LT['frequency'], CDNOW_data_LT['recency'], CDNOW_data_LT['T'])

In [ ]:
# Model summary
bgf.summary

In [ ]:
# Compute the customer alive probability
CDNOW_data_LT['probability_alive'] = bgf.conditional_probability_alive(CDNOW_data_LT['frequency'], CDNOW_data_LT['recency'], CDNOW_data_LT['T'])
CDNOW_data_LT.head(10)

In [ ]:
# Visual representation of relationship between recency and frequency
fig = plt.figure(figsize=(12,8))
plot_probability_alive_matrix(bgf)
plt.show()

In [ ]:
#Predict future transaction for the next 30 days based on historical dataa
t = 30
CDNOW_data_LT['pred_num_txn'] = round(bgf.conditional_expected_number_of_purchases_up_to_time(t, CDNOW_data_LT['frequency'], CDNOW_data_LT['recency'], CDNOW_data_LT['T']),2)
CDNOW_data_LT.sort_values(by='pred_num_txn', ascending=False).head(10).reset_index()

In [ ]:
# Checking the relationship between frequency and monetary_value
CDNOW_return_customers_summary = CDNOW_data_LT[CDNOW_data_LT['frequency']>0]
print(CDNOW_return_customers_summary.shape)
CDNOW_return_customers_summary.head()

In [ ]:
# Checking the relationship between frequency and monetary_value
CDNOW_return_customers_summary[['frequency', 'monetary_value']].corr()

In [ ]:
# Modeling the monetary value using Gamma-Gamma Model
ggf = lifetimes.GammaGammaFitter(penalizer_coef=0.01)
ggf.fit(CDNOW_return_customers_summary['frequency'],
       CDNOW_return_customers_summary['monetary_value'])

In [ ]:
# Summary of the fitted parameters
ggf.summary

In [ ]:
# Calculating the conditional expected average profit for each customer per transaction
CDNOW_data_LT_vF = CDNOW_data_LT[CDNOW_data_LT['monetary_value'] >0]
CDNOW_data_LT_vF['exp_avg_sales'] = ggf.conditional_expected_average_profit(CDNOW_data_LT['frequency'],
                                       CDNOW_data_LT['monetary_value'])
CDNOW_data_LT_vF.head()

In [ ]:
# Checking the expected average value and the actual average value in the data to make sure the values are good
print(f"Expected Average Sales: {CDNOW_data_LT_vF['exp_avg_sales'].mean()}")
print(f"Actual Average Sales: {CDNOW_data_LT_vF['monetary_value'].mean()}")

In [ ]:
# Predicting Customer Lifetime Value for the next 30 days
CDNOW_data_LT_vF['predicted_clv'] =      ggf.customer_lifetime_value(bgf,
                                                               CDNOW_data_LT_vF['frequency'],
                                                               CDNOW_data_LT_vF['recency'],
                                                               CDNOW_data_LT_vF['T'],
                                                               CDNOW_data_LT_vF['monetary_value'],
                                                               time=1,     # lifetime in months
                                                               freq='D',   # frequency in which the data is present(T)      
                                                               discount_rate=0.01) # discount rate
CDNOW_data_LT_vF.head()

In [ ]:
# Manual CLV pridiction
CDNOW_data_LT_vF['manual_predicted_clv'] = CDNOW_data_LT_vF['pred_num_txn'] * CDNOW_data_LT_vF['exp_avg_sales']
CDNOW_data_LT_vF.head()

### TEST

In [ ]:
TEST.head()

In [ ]:
ACTUAL = TEST.groupby("CustomerID")["Price"].sum().reset_index() # Calculating the Sum of Price

ACTUAL.rename(columns ={"Price":"actual_amount_spent_in_next_30_days"}, inplace = True) # renaming the columns

In [ ]:
# Actual vs Predicted Dataframe
Pred_vs_Actual = CDNOW_data_LT_vF.merge(ACTUAL, on = "CustomerID", how = "left")
Pred_vs_Actual.fillna(0, inplace = True)

Pred_vs_Actual.head() # looking at the Actual vs Predicted Dataframe

In [ ]:
# difference between actual and predicted
Pred_vs_Actual["difference"] = Pred_vs_Actual["manual_predicted_clv"] - Pred_vs_Actual["actual_amount_spent_in_next_30_days"]

Pred_vs_Actual.head()

In [ ]:
Pred_vs_Actual["difference"].hist()

In [ ]:
Pred_vs_Actual["difference"].describe()

In [ ]:
Pred_vs_Actual.sort_values('actual_amount_spent_in_next_30_days', ascending = False)

In [ ]:
# MAE of the Lifetimes.
mean_absolute_error(Pred_vs_Actual["actual_amount_spent_in_next_30_days"], Pred_vs_Actual["manual_predicted_clv"])